 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
import requests
import csv
import time
import pandas as pd
import json
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

 Tutaj zdefiniuj paramatry połączenia do API

In [ ]:
APIKEY='*******'

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [ ]:
airports = pd.read_csv(r'..\data\airports.csv')

In [ ]:
airports

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [ ]:
airport_ids = airports['origin_airport_id'].tolist()

In [ ]:
def get_airport_data(airportId, APIKEY):
    url = f'https://api-datalab.coderslab.com/api/v2/airport/{airportId}'
    headers = {'authorization': APIKEY}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print(f'Data for airport Id {airportId} retreived successfully')
        return response.json()
    else:
        print(f'Failed to retrieve data for airport ID {airportId}. Status code: {response.status_code}')
        return None

In [ ]:
with tqdm(total=len(airport_ids), desc='Processing airport data', ncols=100) as pbar:
    for airportId in airport_ids:
        data = get_airport_data(airportId, APIKEY)
        if data:
            airport_data_list.append(data)
        time.sleep(1)

        pbar.update(1)

In [ ]:
airport_df = pd.DataFrame(airport_data_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [ ]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

In [ ]:
airport_df.shape

In [ ]:
airport_df

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [ ]:
airport_df.to_csv('..\\data\\raw\\airport_list.csv', encoding='utf-8', index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [ ]:
def get_weather_data(date, APIKEY):
    url = f'https://api-datalab.coderslab.com/api/v2/airportWeather'    
    params = {'date': date}
    headers = {'Authorization': APIKEY}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        print(f'Data for weather date {date} retreived successfully')
        return response.json()
    else:
        print(f'Failed to retrieve data for weather date {date}. Status code: {response.status_code}')
        return None

In [ ]:
def generate_date_range(start_date, end_date):
    date_range = []
    current_date = start_date
    
    while current_date <= end_date:
        date_range.append(current_date.strftime('%Y-%m')) 
        current_date += timedelta(days=32)
        current_date = current_date.replace(day=1)
    
    return date_range

In [ ]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)
dates = generate_date_range(start_date, end_date)
dates

In [ ]:
weather_data_list = []

for date in dates:
    data_weather = get_weather_data(date, APIKEY)
    if data_weather:
        weather_data_list.append(data_weather)
    time.sleep(1)

In [ ]:
weather_data_list

In [ ]:
airport_weather_long_df = pd.DataFrame(weather_data_list)

In [ ]:
airport_weather_long_df

In [ ]:
expanded_dfs = [pd.json_normalize(airport_weather_long_df[col]) for col in airport_weather_long_df.columns]

In [ ]:
airport_weather_df = pd.concat(expanded_dfs, ignore_index=True)

In [ ]:
airport_weather_df.shape

In [ ]:
airport_weather_df.drop_duplicates(inplace=True)

In [ ]:
airport_weather_df.shape

In [ ]:
airport_weather_df

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [ ]:
airport_weather_df.to_csv('..\\data\\raw\\airport_weather.csv', encoding='utf-8', index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [ ]:
def get_aircraft_data(APIKEY):
    url = f'https://api-datalab.coderslab.com/api/v2/aircraft'
    headers = {'authorization': APIKEY}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print(f'Data for aircraft retreived successfully')
        return response.json()
    else:
        print(f'Failed to retrieve data for aircraft. Status code: {response.status_code}')
        return None

In [ ]:
aircraft_data_list = get_aircraft_data(APIKEY)

In [ ]:
aircraft_df = pd.DataFrame(aircraft_data_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

In [ ]:
aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [ ]:
aircraft_df.to_csv('..\\data\\raw\\aircraft.csv', encoding='utf-8', index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [ ]:
def get_flight_data(airportId, date, APIKEY):
    url = f'https://api-datalab.coderslab.com/api/v2/flight'    
    params = {
        'date': date,
        'airportId': airportId
             }
    headers = {'Authorization': APIKEY}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
def generate_flight_date_range(start_date, end_date):
#WERSJA Z MIESIĄCAMI
    date_range = []
    current_date = start_date

    while current_date <= end_date:
        date_range.append(current_date.strftime('%Y-%m'))
        # Przeskakujemy do następnego miesiąca
        current_date += timedelta(days=32)
        # Przestawiamy datę na pierwszy dzień miesiąca
        current_date = current_date.replace(day=1)

    return date_range

In [ ]:
airport_list = pd.read_csv(r'..\data\raw\airport_list.csv')
airport_list

In [ ]:
test_airportId = airport_list['ORIGIN_AIRPORT_ID'].tolist()
test_airportId

In [ ]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)
dates = generate_flight_date_range(start_date, end_date)
dates

In [ ]:
flight_data_list = []

total_iterations = len(test_airportId) * len(dates)

with tqdm(total=total_iterations) as pbar:
    for airportId in test_airportId:
        for date in dates:
            data_flight = get_flight_data(airportId, date, APIKEY)
            if data_flight:
                flight_data_list.append(data_flight)

            time.sleep(0.5)

            pbar.update(1)

In [ ]:
total_iterations = len(airport_ids) * len(dates)

with tqdm(total=total_iterations) as pbar:
    for airportId in airport_ids:
        for date in dates:
            data_flight = get_flight_data(airportId, date, APIKEY)
            if data_flight:
                flight_data_list.append(data_flight)
            time.sleep(0.5)
            pbar.update(1)

In [ ]:
flight_df_long = pd.DataFrame(flight_data_list)

In [ ]:
expanded_dfs = [pd.json_normalize(flight_df_long[col]) for col in flight_df_long.columns]

In [ ]:
flight_df = pd.concat(expanded_dfs, ignore_index=True)

In [ ]:
flight_df.shape

In [ ]:
flight_df.drop_duplicates(inplace=True)

In [ ]:
flight_df.shape

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [ ]:
flight_df.to_csv('..\\data\\raw\\flight.csv', encoding='utf-8', index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [ ]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)